In [1]:
print('bismillah')

bismillah


In [3]:
#basic modules 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 

from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import AdaBoostRegressor


import warnings


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [3]:
df = pd.read_csv('data/StudentsPerformance.csv')

In [ ]:
df

In [5]:
df.columns = ['gender','race_ethnicity','parental_level_of_education','lunch','test_preparation_course','math_score','reading_score','writing_score']

## Independent and dependent variables 

In [6]:
X = df.drop(columns=['math_score'],axis=1)
y = df['math_score']

In [ ]:
print('categories of "gender" variable :'     ,end=' ')
print(df['gender'].unique())

print('categories of "race_ethnicity" variable :'     ,end=' ')
print(df['race_ethnicity'].unique())

print(f"categoreis of 'parental_level_of_education' :  {df['parental_level_of_education'].unique()}")

print('categories of "lunch" variable :'     ,end=' ')
print(df['lunch'].unique())

print('categories of "test_preparation_course" variable :'     ,end=' ')
print(df['test_preparation_course'].unique())

## Column transformer 

In [8]:
numerical_feature = X.select_dtypes(exclude='object').columns
categorical_feature =  X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

oh_encoder = OneHotEncoder()
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder',oh_encoder,categorical_feature),
        ('StandardScaler',numerical_transformer,numerical_feature)
    ]
)

In [9]:
X = preprocessor.fit_transform(X)

In [ ]:
X.shape

### train and test split 

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape 

### Performance Metrix Evaluation Function 

In [12]:
def evaluate(true,predict):
    mae = mean_absolute_error(true,predict)
    mse = mean_squared_error(true,predict)
    rmse = np.sqrt(mean_squared_error(true,predict))
    r2 = r2_score(true,predict)
    return mse,mae,rmse,r2

## Model Training 

In [ ]:
models = {
    'Linear Regression' : LinearRegression(),
    'Lasso' : Lasso(),
    'Ridge' : Ridge(),
    'K-Nearest neighbour' : KNeighborsRegressor(),
    'Decision Tree' : DecisionTreeRegressor(),
    'Random Forest' : RandomForestRegressor(),
    'XgBoost Regressor' : XGBRegressor(),
    'CatBoost Regressor' : CatBoostRegressor(),
    'AdaBoost Regressor' : AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    #prediction 
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #evaluate 
    train_mse, train_mae, train_rmse, train_r2 = evaluate(y_train,y_train_pred)
    test_mse, test_mae, test_rmse, test_r2 = evaluate(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training data')
    print('- Mean Absolute error : {:.4f}'.format(train_mae))
    print('- Mean Sqaure error : {:.4f}'.format(train_mse))
    print('- Root Mean Square error  : {:.4f}'.format(train_rmse))
    print('- R2_score : {:.4f}'.format(train_r2))

    print('------------------------------------------') 

    print('Model performance for Testing data')
    print('- Mean Absolute error : {:.4f}'.format(test_mae))
    print('- Mean Sqaure error : {:.4f}'.format(test_mse))
    print('- Root Mean Square error  : {:.4f}'.format(test_rmse))
    print('- R2_score : {:.4f}'.format(test_r2))
    r2_list.append(test_r2)

    print('='*32)


## Result 

In [ ]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model_name','accuracy']).sort_values(by=['accuracy'],ascending=False)

In [ ]:
lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

## y_pred and y_test

In [ ]:
plt.scatter(y_test,y_pred);
plt.xlabel('Actual');
plt.ylabel('Predicted');

In [ ]:
sns.regplot(x=y_test,y=y_pred,ci=None,color ='red');

## diffrence between actual and redicted values 

In [ ]:
pred_df=pd.DataFrame({'Actual Value':y_test,'Predicted Value':y_pred,'Difference':y_test-y_pred})
pred_df